In [ ]:
from wayne_utils import load_data, save_data
import os
import sys
from tools.utils import points_dict, get_chunk

teams = ["home team", "away team"]

_Data_PATH = "workspace/T-Tuple-T"

test_path = os.path.join( _Data_PATH, "data/test.json")
train_path = os.path.join( _Data_PATH, "data/train.json")

_ROOT_PATH = "workspace/TKGT"
test_list = load_data( test_path, "json")           # 754
train_list = load_data( train_path, "json")         # 3017

score_instruction = """You are an information extraction expert, please extract the football game final scores from context.
"""
score_input = """You should only return the scores of home team vs away team in format like "1-2" without other word.
Context: {Context}
Question: What is the final scores?
Answer:
"""

other_instruction = """You are an information extraction expert, please judge whether the home or away team has scored at a certain technical point according to the context.
You should only return 'yes' or 'no'.
"""
other_input = """Context: {Context}
Question: Does the {Team} have a {Field}?
Answer:
"""
def construct_prompt(  context, answer , score = True, team = None, field = None):
    temp = {}
    if score:
        temp["instruction"] = score_instruction
        temp["input"] = score_input.format( Context = context)
    else:
        temp["instruction"] = other_instruction
        temp["input"] = other_input.format( Context = context, Team = team, Field = field )
    temp["output"] = answer
    return temp

# 下面构建一个文档的prompt

## 构建比分提示词

In [124]:
def get_vs_from_table( table ):
    post_ = table.split( "Away Team,")[1]
    Away_Team = post_.split( "<NEWLINE>Home Team,")[0].strip()
    Home_Team = post_.split( "<NEWLINE>Home Team,")[1].strip()
    home_score = Home_Team.split( ",")[0]
    away_score = Away_Team.split( ",")[0]
    return f"{home_score}-{away_score}"

def get_vs_score_prompt( text_list, table ):
    vs_output = get_vs_from_table( table )
    vs_context = text_list[-2: ]
    _prompt = construct_prompt( vs_context, vs_output)
    return _prompt

# 追加保存
def append_ft( _new_json ):
    ft_path = os.path.join( _ROOT_PATH, "ft_data.json")
    json_data = load_data( ft_path , "json")
    json_data.append( _new_json )
    print( f"当前长度{len(json_data)}")
    save_data( json_data, ft_path)

vs_ft_list = [] 
for index in range(200):
    text = train_list[index]['text']
    table = train_list[index]['table']
    text_list = get_chunk( text )
    vs_prompt = get_vs_score_prompt( text_list, table )
    vs_ft_list.append( vs_prompt )

save_data( vs_ft_list, os.path.join( _ROOT_PATH, "test/LiveSum/ft_data_vs.json"))

## 构建其他指标提示词

In [29]:
def get_other_prompt( text_list ):
    single_prompts = []
    for sub_index in range(len(text_list)):
        line = text_list[sub_index].lower()
        for team_name in teams:
            if team_name in line:
                for key in points_dict.keys():
                    for point in points_dict[key]:
                        if point in line:
                            # 队名和字段都在
                            context = text_list[sub_index]
                            answer = ""
                            prompt = construct_prompt(  context, answer , False, team = team_name, field = key)
                            single_prompts.append( prompt )
                            break
    return single_prompts

other_ft_list = []
index = 500
text = train_list[index]['text']
table = train_list[index]['table']
text_list = get_chunk( text )

single_prompts = get_other_prompt( text_list )

In [31]:
answer_0 = [                                                                        # 111
    "yes", "no", "no", "yes", "yes", "yes", "yes", "yes", "yes", "yes",             # 0:10
    "yes", "no", "yes", "yes", "no", "yes", "yes", "yes", "yes", "yes", 
    "yes", "yes", "yes", "yes", "yes", "yes", "yes", "no", "no", "yes", 
    "yes", "yes", "no", "yes", "no", "yes", "yes", "yes", "yes", "yes", 
    "yes", "yes", "yes", "no", "yes", "yes", "no", "yes", "yes", "no", 
    "yes", "no", "no", "no", "yes", "no", "no", "yes", "yes", "yes",                # 50:60
    "no", "no", "yes", "yes", "yes", "yes", "yes", "yes", "yes", "yes",
    "yes", "yes", "yes", "no", "yes", "no", "no", "yes", "yes", "yes",
    "yes", "yes", "yes", "yes", "yes", "yes", "no", "yes", "yes", "no",
    "yes", "yes", "yes", "yes", "yes", "yes", "yes", "no", "no", "yes",           # 90:100
    "no", "yes", "yes", "no", "yes", "yes", "yes", "yes", "no", "no",
    "yes",
]

In [50]:
answer_500 = [                                                                        # 119
    "yes", "no", "no", "yes", "yes", "yes", "yes", "yes", "yes", "no",             # 0:10
    "yes", "yes", "no", "yes", "yes", "yes", "yes", "yes", "yes", "yes",
    "yes", "no", "no", "yes", "no", "yes", "yes", "no", "yes", "yes",
    "yes", "no", "yes", "yes", "no", "yes", "yes", "yes", "yes", "yes",          # 30:40
    "yes", "yes", "yes", "yes", "yes", "yes", "yes", "yes", "no", "no",
    "yes", "yes", "yes", "yes", "no", "yes", "yes", "no", "yes", "yes", 
    "yes", "yes", "yes", "no", "yes", "yes", "no", "yes", "no", "no",        # 60:70
    "yes", "yes", "yes", "yes", "yes", "yes", "no", "no", "yes", "yes",
    "yes", "yes", "yes", "yes", "yes", "yes", "yes", "yes", "no", "yes",
    "yes", "no", "yes", "no", "yes", "yes", "no", "yes", "yes", "yes",
    "yes", "yes", "yes", "no", "yes", "yes", "yes", "yes", "yes", "yes",    # 100:110
    "no", "yes", "yes", "no", "yes", "yes", "yes", "no", "no",
]

In [47]:
def show_input( single_prompts ):
    for i in range( len(single_prompts) ):
        print(i+1)
        print(single_prompts[i]["input"]  )


show_input( single_prompts[110:120] )

1
Context: Player8(Home Team) earns a free kick on the left wing after a foul by Player34(Away Team)
Question: Does the home team have a foul?
Answer:

2
Context: Player8(Home Team) earns a free kick on the left wing after a foul by Player34(Away Team)
Question: Does the home team have a free kick?
Answer:

3
Context: Player8(Home Team) earns a free kick on the left wing after a foul by Player34(Away Team)
Question: Does the away team have a foul?
Answer:

4
Context: Player8(Home Team) earns a free kick on the left wing after a foul by Player34(Away Team)
Question: Does the away team have a free kick?
Answer:

5
Context: Player4(Home Team) of the Home Team attempts a through ball, but Player8(Home Team) is offside
Question: Does the home team have a offside?
Answer:

6
Context: Player28(Away Team) misses the target with a high right-footed shot from outside the box, assisted by Player31(Away Team)
Question: Does the away team have a shot?
Answer:

7
Context: Player28(Away Team) misses 

In [52]:
def ans( single_prompts, answer ):
    if len( single_prompts) != len(answer):
        raise Exception( "长度不一致")
    for i in range( len(single_prompts) ):
        single_prompts[i]["output"] = answer[i]
    return single_prompts

new_single_prompts = ans( single_prompts, answer_500 )
save_data( new_single_prompts, os.path.join( _ROOT_PATH, "test/LiveSum/ft_data_500.json"))

In [59]:
merge_ft = []
ft_data_500 = load_data( os.path.join( _ROOT_PATH, "test/LiveSum/ft_data_500.json"), "json")
ft_data_1 = load_data( os.path.join( _ROOT_PATH, "test/LiveSum/ft_data_1.json"), "json")
ft_data_vs = load_data( os.path.join( _ROOT_PATH, "test/LiveSum/ft_data_vs.json"), "json")

merge_ft.extend( ft_data_500 )
merge_ft.extend( ft_data_1 )
merge_ft.extend( ft_data_vs )

In [61]:
len(merge_ft)

430

In [62]:
save_data( merge_ft, os.path.join( _ROOT_PATH, "test/LiveSum/ft_data_all_430.json"))